In [15]:
import cv2
import math


def video_to_frames(video_path, save_path=None, num_frames=12, crop=(150, 200, 100, 100)):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame interval
    frame_interval = math.floor(total_frames / num_frames)

    # Initialize an empty list to store the frames
    frames = []

    # Loop through the video frames and extract the frames at the specified interval
    for i in range(0, total_frames, frame_interval):
        # Set the frame index
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        # Read the frame
        ret, frame = cap.read()
        # Remove the white border (assuming a 5-pixel border)
        # frame = cv2.copyMakeBorder(frame, 30, 20, 20, 30, cv2.BORDER_CONSTANT, value=(0, 0, 0))

        h, w, _ = frame.shape
        cropped = frame[(h-crop[0])//2:(h-crop[1])//2+crop[1], (w-crop[2])//2:(w-crop[3])//2+crop[3], :]
        # Add the frame to the list
        frames.append(cropped)
    
    # Concatenate the frames horizontally
    frame_array = cv2.hconcat(frames)

    # Save the concatenated frames as a single image with higher resolution
    save_path = save_path and video_path.replace('.mp4', '.png')
    cv2.imwrite(save_path, frame_array)


In [16]:
file_path = '/home/ctq566/motion-diffusion-model/save/in_thesis/sample16_rep00.mp4'
video_to_frames(file_path, save_path='/home/ctq566/motion-diffusion-model/save/in_thesis/sample16_rep00.png')